# Reproducing MCCD's Experiments

In [256]:
from functools import partial
from io import StringIO

from surface_sim.setups.setup import SetupDict

from mccd.random_clifford_circuit import *
from surface_sim.setups import CircuitNoiseSetup
from surface_sim.models import CircuitNoiseModel, BiasedCircuitNoiseModel
from surface_sim import Detectors, Setup
from surface_sim.experiments import schedule_from_circuit, experiment_from_schedule
import time
import stim

from pathlib import Path
import stim
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from joblib import Parallel, delayed
import itertools
import shelve
from surface_sim.layouts import rot_surface_codes

from pymatching import Matching as MWPM
from mle_decoder import MLEDecoder as MLE
from stimbposd import BPOSD
from sklearn.metrics import accuracy_score


MWPM. We use the open-source library PyMatching42 with the noise model used for data generation as detailed in the ‘Experimentally motivated noise model’ subsection.

BP-OSD. We use the open-source library stimbposd43. We use the exact noise model used for data generation and set the maximal belief propagation iterations to 20.

MLE. We use the algorithm developed and implemented as in ref. 14.

In [257]:
DECODER_BASELINES = {
    'BPOSD': partial(BPOSD, max_bp_iters=20),
    'MLE': MLE,
    'MWPM': MWPM,
}

In [18]:
from surface_sim.circuit_blocks.rot_surface_code_css import gate_to_iterator
print('Rotated', gate_to_iterator.keys())
from surface_sim.circuit_blocks.unrot_surface_code_css import gate_to_iterator
print('Unrotated', gate_to_iterator.keys())
ROT_GATES = list('IXZ')
UNR_GATES = list('IHXZ')
MCCD_GATES = ['I', 'X', 'Y', 'Z', 'H']

Rotated dict_keys(['TICK', 'I', 'S', 'X', 'Z', 'CX', 'CNOT', 'R', 'RZ', 'RX', 'M', 'MZ', 'MX'])
Unrotated dict_keys(['TICK', 'I', 'S', 'H', 'X', 'Z', 'CX', 'CNOT', 'R', 'RZ', 'RX', 'M', 'MZ', 'MX'])


In [239]:
def to_stim_circuit(mccd_circuit):
    res = stim.Circuit()
    # Must have R and M. Error inactive layout.
    for n in range(mccd_circuit.n_logical_qubits):
        res.append('R', [n])

    for name, timestep, qubits in mccd_circuit:
        res.append(name, qubits)
        # No need to add TICK here. surface-sim will add for us.

    for n in range(mccd_circuit.n_logical_qubits):
        res.append('M', [n])

    return res

def print_random_circuit(c: RandomCliffordCircuit):
    return list(c)

def dict_product(input_dict):
    keys = input_dict.keys()
    value_lists = input_dict.values()

    # 使用itertools.product生成所有值的组合
    value_combinations = itertools.product(*value_lists)

    # 将每个值的组合与键配对，生成字典列表
    for combo in value_combinations:
        yield dict(zip(keys, combo))

def run_decoder(name: str, circuit: stim.Circuit, shots: int):
    method = DECODER_BASELINES[name](circuit.detector_error_model())
    sampler = circuit.compile_detector_sampler()
    syndrome, labels = sampler.sample(shots=shots, separate_observables=True)
    begin = time.time_ns()
    predictions = method.decode_batch(syndrome)
    end = time.time_ns()
    logical_accuracy = accuracy_score(labels, predictions)
    walltime_seconds = (end - begin) / 1e9
    return dict(
        decoder=name,
        logical_accuracy=logical_accuracy,
        walltime_seconds=walltime_seconds,
    )

def compile_to_physical(log_cir: stim.Circuit, distance: int, setup_dict: SetupDict,
                        rotated=True) -> stim.Circuit:
    if rotated:
        from surface_sim.circuit_blocks.rot_surface_code_css import gate_to_iterator
        from surface_sim.layouts import rot_surface_codes
        layouts = rot_surface_codes(log_cir.num_qubits, distance=distance)
    else:
        from surface_sim.circuit_blocks.unrot_surface_code_css import gate_to_iterator
        from surface_sim.layouts import unrot_surface_codes
        layouts = unrot_surface_codes(log_cir.num_qubits, distance=distance)

    setup = CircuitNoiseSetup()
    setup.set_var_param("prob", 1e-3)
    model = CircuitNoiseModel.from_layouts(setup, *layouts)
    detectors = Detectors.from_layouts("pre-gate", *layouts)
    schedule = schedule_from_circuit(log_cir, layouts, gate_to_iterator)
    phy_cir: stim.Circuit = experiment_from_schedule(
        schedule, model, detectors, anc_reset=True
    )
    return phy_cir


def random_mirror_symmetric_clifford(n_logical_qubits: int, circuit_index: str, depth: int, rotated=True) -> stim.Circuit:
    logical_class = TypeICircuit if circuit_index == '3' else TypeIICircuit
    random_circuit = logical_class(
        n_logical_qubits=n_logical_qubits,
        circuit_index=circuit_index,
        depth=depth,
        single_qubit_gate_list=ROT_GATES if rotated else ROT_GATES,
    )
    random_circuit.sample_circuit()
    return to_stim_circuit(random_circuit)


## Circuit Depths

Finally, to ensure that MCCD’s training generalizes to new circuits, we train MCCD with data from logical circuit depth D = 2, 4, 6, 8 and 10 (D = 4, 8, 12, 16 and 20) and test the model on a wide range of unseen circuits with depth up to D = 18 (D = 36) for Circuit Type I (Circuit Type II), defined below.

In [244]:
n_logical_qubits=2
circuit_index = '4'
depth = 2
code_distance=3 # 2 is not ok with rot code.
num_shots = 1000
num_circuits = 100


In [206]:
logical_class = TypeICircuit if circuit_index == '3' else TypeIICircuit

random_circuit = logical_class(
    n_logical_qubits=n_logical_qubits,
    circuit_index=circuit_index,
    depth=depth,
    single_qubit_gate_list=ROT_GATES)

random_circuit.sample_circuit()

In [207]:
print_random_circuit(random_circuit)

[(np.str_('X'), 0, [0]),
 (np.str_('I'), 0, [1]),
 (np.str_('I'), 1, [0]),
 (np.str_('I'), 1, [1]),
 (np.str_('I'), 1, [0]),
 (np.str_('I'), 1, [1]),
 (np.str_('X'), 0, [0]),
 (np.str_('I'), 0, [1])]

In [153]:
random_circuit.n_logical_qubits

2

In [227]:
CircuitNoiseSetup().to_dict()

{'name': 'Circuit-level noise setup',
 'description': 'Setup for a circuit-level noise model that can be used for any code and distance.',
 'gate_durations': {},
 'setup': [{'sq_error_prob': '{prob}',
   'tq_error_prob': '{prob}',
   'meas_error_prob': '{prob}',
   'reset_error_prob': '{prob}',
   'idle_error_prob': '{prob}',
   'assign_error_flag': False,
   'assign_error_prob': '{prob}'}]}

In [233]:
log_cir = to_stim_circuit(random_circuit)

In [209]:
print(log_cir)

R 0 1
X 0
I 1 0 1 0 1
X 0
I 1
M 0 1


### Noise Model

Experimentally motivated noise model. For the numerical studies presented in the ‘Results’ section, we use the stim package for simulation. We consider a circuit-level noise model motivated by the current experimental capability of neutral atom array-based quantum computers. Specifically, we use a circuit-level noise model that includes the following physical noises:

• Each two-qubit physical gate is followed by a two-qubit Pauli noise channel with probability [0.0005, 0.00175, 0.000625, 0.0005, 0, 0, 0, 0.00175, 0, 0, 0, 0.000625, 0, 0, 0.00125]

• Each single-qubit physical gate is followed by a single-qubit depolarizing model with probability [0.0001, 0.0001, 0.0001]

• On a physical level, the atoms are moved to achieve flexible connectivity between different physical qubits. This comes at the cost of having idling error due to the extra time taken during the physical qubit movement, which is captured as a Pauli noise channel with probability [4 × 10−7, 4 × 10−7, 1.6 × 10−6]. This error channel is applied when physical qubit movement happens.

• Resetting a physical qubit has a bit flip error probability of P = 0.002.

• Measuring a physical qubit has a bit flip error probability of P = 0.002.



In [202]:
tq_noise = sum([0.0005, 0.00175, 0.000625, 0.0005, 0, 0, 0, 0.00175, 0, 0, 0, 0.000625, 0, 0, 0.00125])
sq_noise = sum([0.0001, 0.0001, 0.0001])
reset_noise = 0.002
measure_noise = 0.002
idle_noise = sum([4 * 1e-7, 4 * 1e-7, 1.6 * 1e-6])
SETUP_DICT = {
    "setup": [{
        "sq_error_prob": sq_noise,
        "tq_error_prob": tq_noise,
        "meas_error_prob": measure_noise,
        "reset_error_prob": reset_noise,
        "idle_error_prob": idle_noise,
        "assign_error_flag": False,
        "assign_error_prob": 0.0,
    }],
    "name": "MCCD",
    "description": "MCCD paper experimental noise",
}

In [203]:
tq_noise, sq_noise, reset_noise, measure_noise, idle_noise

(0.007, 0.00030000000000000003, 0.002, 0.002, 2.4e-06)

In [236]:

layouts = rot_surface_codes(log_cir.num_qubits, distance=code_distance)
# setup = Setup(SETUP_DICT)
setup = CircuitNoiseSetup()
setup.set_var_param("prob", 1e-3)
model = CircuitNoiseModel.from_layouts(setup, *layouts)
detectors = Detectors.from_layouts("pre-gate", *layouts)

schedule = schedule_from_circuit(log_cir, layouts, gate_to_iterator)
phy_cir: stim.Circuit = experiment_from_schedule(
    schedule, model, detectors, anc_reset=True
)

In [237]:
setup.to_dict()

{'name': 'Circuit-level noise setup',
 'description': 'Setup for a circuit-level noise model that can be used for any code and distance.',
 'gate_durations': {},
 'setup': [{'sq_error_prob': '{prob}',
   'tq_error_prob': '{prob}',
   'meas_error_prob': '{prob}',
   'reset_error_prob': '{prob}',
   'idle_error_prob': '{prob}',
   'assign_error_flag': False,
   'assign_error_prob': '{prob}'}]}

In [238]:
print(phy_cir)

QUBIT_COORDS(0, 0) 0
QUBIT_COORDS(0, 2) 1
QUBIT_COORDS(0, 4) 2
QUBIT_COORDS(2, 0) 3
QUBIT_COORDS(2, 2) 4
QUBIT_COORDS(2, 4) 5
QUBIT_COORDS(4, 0) 6
QUBIT_COORDS(4, 2) 7
QUBIT_COORDS(4, 4) 8
QUBIT_COORDS(-1, 1) 9
QUBIT_COORDS(1, 3) 10
QUBIT_COORDS(3, 1) 11
QUBIT_COORDS(5, 3) 12
QUBIT_COORDS(1, 1) 13
QUBIT_COORDS(1, 5) 14
QUBIT_COORDS(3, -1) 15
QUBIT_COORDS(3, 3) 16
QUBIT_COORDS(0, 8) 17
QUBIT_COORDS(0, 10) 18
QUBIT_COORDS(0, 12) 19
QUBIT_COORDS(2, 8) 20
QUBIT_COORDS(2, 10) 21
QUBIT_COORDS(2, 12) 22
QUBIT_COORDS(4, 8) 23
QUBIT_COORDS(4, 10) 24
QUBIT_COORDS(4, 12) 25
QUBIT_COORDS(-1, 9) 26
QUBIT_COORDS(1, 11) 27
QUBIT_COORDS(3, 9) 28
QUBIT_COORDS(5, 11) 29
QUBIT_COORDS(1, 9) 30
QUBIT_COORDS(1, 13) 31
QUBIT_COORDS(3, 7) 32
QUBIT_COORDS(3, 11) 33
R 0 1 2 3 4 5 6 7 8 17 18 19 20 21 22 23 24 25
X_ERROR(0.001) 0 1 2 3 4 5 6 7 8 17 18 19 20 21 22 23 24 25
I 9 10 11 12 13 14 15 16 26 27 28 29 30 31 32 33
DEPOLARIZE1(0.001) 9 10 11 12 13 14 15 16 26 27 28 29 30 31 32 33
TICK
I 5 13 7 10 1 15 4 0 1

In [108]:
sampler = phy_cir.compile_detector_sampler()

In [109]:
syn, labels = sampler.sample(shots=10, separate_observables=True)

In [110]:
syn.shape

(10, 8)

In [111]:
syn.reshape((10, n_logical_qubits, depth, -1)).shape

(10, 2, 2, 2)

In [120]:
print(phy_cir)

QUBIT_COORDS(0, 0) 0
QUBIT_COORDS(0, 2) 1
QUBIT_COORDS(0, 4) 2
QUBIT_COORDS(2, 0) 3
QUBIT_COORDS(2, 2) 4
QUBIT_COORDS(2, 4) 5
QUBIT_COORDS(4, 0) 6
QUBIT_COORDS(4, 2) 7
QUBIT_COORDS(4, 4) 8
QUBIT_COORDS(-1, 1) 9
QUBIT_COORDS(1, 3) 10
QUBIT_COORDS(3, 1) 11
QUBIT_COORDS(5, 3) 12
QUBIT_COORDS(1, 1) 13
QUBIT_COORDS(1, 5) 14
QUBIT_COORDS(3, -1) 15
QUBIT_COORDS(3, 3) 16
QUBIT_COORDS(0, 8) 17
QUBIT_COORDS(0, 10) 18
QUBIT_COORDS(0, 12) 19
QUBIT_COORDS(2, 8) 20
QUBIT_COORDS(2, 10) 21
QUBIT_COORDS(2, 12) 22
QUBIT_COORDS(4, 8) 23
QUBIT_COORDS(4, 10) 24
QUBIT_COORDS(4, 12) 25
QUBIT_COORDS(-1, 9) 26
QUBIT_COORDS(1, 11) 27
QUBIT_COORDS(3, 9) 28
QUBIT_COORDS(5, 11) 29
QUBIT_COORDS(1, 9) 30
QUBIT_COORDS(1, 13) 31
QUBIT_COORDS(3, 7) 32
QUBIT_COORDS(3, 11) 33
R 0 1 2 3 4 5 6 7 8 17 18 19 20 21 22 23 24 25
X_ERROR(0.001) 0 1 2 3 4 5 6 7 8 17 18 19 20 21 22 23 24 25
I 9 10 11 12 13 14 15 16 26 27 28 29 30 31 32 33
DEPOLARIZE1(0.001) 9 10 11 12 13 14 15 16 26 27 28 29 30 31 32 33
TICK
I 5 13 7 10 1 15 4 0 1

In [122]:
log_cir.diagram()

q0: -R-X-X-M:rec[0]-

q1: -R-I-I-M:rec[1]-

In [136]:
html = phy_cir.diagram("timeline-3d-html")

In [140]:
Path('./timeline3d.html').write_text(str(html))

193084

In [218]:
def generate_circuit(distance: int, depth: int, circuit_index: str):
    n_logical_qubits = 2 if circuit_index == '4' else 1
    log_cir = random_mirror_symmetric_clifford(n_logical_qubits, circuit_index, depth)
    phy_cir: stim.Circuit = compile_to_physical(log_cir, distance, SETUP_DICT)
    config = dict(distance=distance, depth=depth, circuit_type_index=circuit_index)
    return phy_cir, log_cir, config


In [245]:
n_logical_qubits=1
circuit_index = '4'
depth = 5
code_distance=3 # 2 is not ok with rot code.
num_shots = 1000
num_circuits = 10

for depth in [4, 8, 12, 16, 20, 24, 28, 32, 36]:
    phy_cir, *_ = generate_circuit(distance=code_distance, depth=depth,
                                   circuit_index=circuit_index)
    res = run_decoder('BPOSD', phy_cir, shots=num_shots)
    print(res)


{'decoder': 'BPOSD', 'logical_accuracy': 0.96, 'walltime_seconds': 0.003709}
{'decoder': 'BPOSD', 'logical_accuracy': 0.925, 'walltime_seconds': 0.004874}
{'decoder': 'BPOSD', 'logical_accuracy': 0.851, 'walltime_seconds': 0.006625}
{'decoder': 'BPOSD', 'logical_accuracy': 0.821, 'walltime_seconds': 0.008294}
{'decoder': 'BPOSD', 'logical_accuracy': 0.778, 'walltime_seconds': 0.010214}
{'decoder': 'BPOSD', 'logical_accuracy': 0.702, 'walltime_seconds': 0.01268}
{'decoder': 'BPOSD', 'logical_accuracy': 0.662, 'walltime_seconds': 0.015596}
{'decoder': 'BPOSD', 'logical_accuracy': 0.608, 'walltime_seconds': 0.018511}
{'decoder': 'BPOSD', 'logical_accuracy': 0.562, 'walltime_seconds': 0.021799}


## Reproduce Baselines

### Shots & Repeat
We evaluate each decoder over 20 independent runs. In each run, we randomly sample 1,000 syndrome trajectories from Type I circuits and average them to obtain a run-level performance estimate. We report the mean across the 20 runs, with error bars showing s.e.m.


In [246]:
num_shots = 1000
trial = 20

def get_figure4_settings():
    distance = [3, 5]
    depth = [ 2,  4,  6,  8, 10, 12, 14, 16, 18]
    # Type I one qubit. Type II two qubits.
    circuit_index = ['3']

    return dict_product(dict(
        distance=distance,
        depth=depth,
        circuit_index=circuit_index,
    ))

def get_figure5_settings():
    distance = [3, 5]
    depth = [ 4,  8, 12, 16, 20, 24, 28, 32, 36]
    # Type I one qubit. Type II two qubits.
    circuit_index = ['4']

    return dict_product(dict(
        distance=distance,
        depth=depth,
        circuit_index=circuit_index,
    ))

figure4_settings = get_figure4_settings()
figure5_settings = get_figure5_settings()

In [191]:
def get_syndrome_labels(circuit: stim.Circuit, shots: int, depth: int, n_logical_qubits: int):
    sampler = circuit.compile_detector_sampler()
    syndrome, labels = sampler.sample(shots=shots, separate_observables=True)
    print(syndrome.shape, labels.shape)
    syndrome = syndrome.reshape((shots, n_logical_qubits, depth * 2, -1))
    return syndrome, labels

In [247]:
def generate_circuit_tasks(baselien_settings):

    def gen_circuit_to_str(**params):
        phy_cir, log_cir, config = generate_circuit(**params)
        return str(phy_cir), str(log_cir), config

    for params in baselien_settings:
        yield delayed(gen_circuit_to_str)(**params)

fig4_circuits = Parallel(n_jobs=-1, verbose=1)(generate_circuit_tasks(figure4_settings))
fig5_circuits = Parallel(n_jobs=-1, verbose=1)(generate_circuit_tasks(figure5_settings))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not i

In [248]:
# Must save the circuits for fair comparison with MCCD.
# Filename format: d3_c3_D1, distance=3, circuit_index=3, depth=1

def save_circuits(subdir, bench_circuits):
    save_dir = Path('./data/bench/circuits') / subdir
    save_dir.mkdir(parents=True, exist_ok=True)

    for phy_cir, log_cir, config in bench_circuits:
        distance, depth, circuit_index = config['distance'], config['depth'], config['circuit_type_index']
        filename = f'd{distance}_c{circuit_index}_D{depth}'
        (save_dir / (filename + '_phy')).with_suffix('.stim').write_text(phy_cir)
        (save_dir / (filename + '_log')).with_suffix('.stim').write_text(log_cir)

In [253]:
def run_decoder_tasks(bench_circuits, df_name):

    def run_decoder_plus(config, cir_str, **kwargs):
        cir = stim.Circuit.from_file(StringIO(cir_str))
        kwargs['circuit'] = cir
        res = run_decoder(**kwargs)
        res.update(config)
        return res

    def tasks():
        for phy_cir, _, config in bench_circuits:
            for decoder in DECODER_BASELINES.keys():
                for t in range(trial):
                    yield delayed(run_decoder_plus)(config, phy_cir,
                                                    name=decoder, shots=num_shots)

    bench_result = Parallel(n_jobs=-1, verbose=1)(tasks())

    df = pd.DataFrame.from_records(bench_result)
    df = pd.melt(df, id_vars=['decoder', 'distance', 'depth', 'circuit_type_index'],
             value_vars=['walltime_seconds', 'logical_accuracy'],
             var_name='metric',
             value_name='value')

    filename = Path(f'./data/result/{df_name}.csv')
    filename.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(filename, index=False)
    print('done')
    return df


In [254]:
df4 = run_decoder_tasks(fig4_circuits, 'reproduce_fig4')

df5 = run_decoder_tasks(fig5_circuits, 'reproduce_fig5')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 917 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:   36.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 246 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done 530 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 880 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  5.6min finished
